In [23]:
import numpy as np
import tensorflow as tf
import helpers
tf.reset_default_graph()
sess = tf.InteractiveSession()

encoder_inputs = tf.placeholder(shape = (None, None), dtype = tf.int32)
decoder_targets = tf.placeholder(shape = (None, None), dtype = tf.int32)
decoder_inputs = tf.placeholder(shape = (None, None), dtype = tf.int32)

In [38]:
#get in the data
chinese = []
english_phoeneme = []
with open('dataset.txt') as f:
    for line in f:
        temp = line.split('\t')
        chinese.append(temp[1])
        english_phoeneme.append(temp[2])
english_phoeneme = [phoeneme.split('\n')[0].split() for phoeneme in english_phoeneme]
chinese = [list(word) for word in chinese]


In [24]:
def prepare_vocab_input():
    vocab_inputs = []
    with open('./english_phoneme_vocabulary_output.txt') as file:
        for line in file:
            vocab_inputs.append(line.split('\n')[0])
    vocab_inputs.remove('_PAD')
    vocab_inputs.remove('_GO')
    vocab_inputs.remove('_EOS')
    vocab_inputs.remove('_UNK')
    vocab_inputs = ['PAD', 'EOS'] + (vocab_inputs)
    return vocab_inputs

In [25]:
def prepare_vocab_predict():
    vocab_predict = list(chinese_id_dict.keys())
    return vocab_predict

In [26]:
# prepare chinese_id_dict
chinese_id_list = []
with open('./chinese_vocabulary.txt') as file7:
    for line in file7:
        chinese_id_list.append(line.split('\n')[0])
chinese_id_list = ['PAD', 'EOS'] + chinese_id_list
chinese_id_dict = {}
for i in range(len(chinese_id_list)):
    chinese_id_dict[chinese_id_list[i]] = i
# finish preparation for chinese_id_dict

In [ ]:

english_phoneme_dict = {}
for i in range(len(vocab_inputs)):
    english_phoneme_dict[vocab_inputs[i]] = i

In [27]:
vocab_inputs = prepare_vocab_input()
vocab_predict = prepare_vocab_predict()

In [28]:
greatestvalue_predict = 42
encoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_inputs), greatestvalue_predict]
                                                   , -1.0, 1.0), dtype = tf.float32)

decoder_embeddings = tf.Variable(tf.random_uniform([len(vocab_predict), greatestvalue_predict]
                                                   , -1.0, 1.0), dtype = tf.float32)


encoder_inputs_embedded = tf.nn.embedding_lookup(encoder_embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(decoder_embeddings, decoder_inputs)

### Encoder

In [29]:
# RNN size of greatestvalue_inputs
encoder_cell = tf.contrib.rnn.LSTMCell(greatestvalue_predict)

# 2 layers of RNN
#encoder_rnn_cells =tf.contrib.rnn.MultiRNNCell([encoder_cell] * 2)

_, encoder_final_state = tf.nn.dynamic_rnn(encoder_cell, encoder_inputs_embedded,
                                           dtype = tf.float32, time_major = True)

### Decoder

In [30]:
# RNN size of greatestvalue_predict
decoder_cell = tf.contrib.rnn.LSTMCell(greatestvalue_predict)

# 2 layers of RNN
#decoder_rnn_cells = tf.nn.rnn_cell.MultiRNNCell([decoder_cell] * 2)

# declare a scope for our decoder, later tensorflow will confuse
decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(decoder_cell, decoder_inputs_embedded, 
                                                         initial_state = encoder_final_state,
                                           dtype = tf.float32, time_major = True, scope ='decoder')

In [31]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, len(vocab_predict))

decoder_prediction = tf.argmax(decoder_logits, 2)

In [32]:
# this might very costly if you have very large vocab
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels = tf.one_hot(decoder_targets, depth = len(vocab_predict), dtype = tf.float32),
    logits = decoder_logits)

loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

sess.run(tf.global_variables_initializer())

In [62]:
batch_size = 4000
epoch = 100
LOSS = []

def feeding(inputs, labels):
    inputs_int = []; predict_int = []
    for i in range(len(inputs)):
        single_input = []
        single_predict = []
        for x in range(len(labels[i])):
            try:
                single_input += [english_phoneme_dict[inputs[i][x]]]
            except:
                single_input += [0]
        for x in range(len(labels[i])):
            single_predict += [chinese_id_dict[labels[i][x]]]
        inputs_int.append(single_input); predict_int.append(single_predict)
            
    enc_input, _ = helpers.batch(inputs_int)
    dec_target, _ = helpers.batch([(sequence) + [1] for sequence in predict_int])
    dec_input, _ = helpers.batch([[1] + (sequence) for sequence in inputs_int])
    
    return {encoder_inputs: enc_input, decoder_inputs: dec_input, decoder_targets: dec_target}

In [63]:
import time

for q in range(epoch):
    total_loss = 0
    lasttime = time.time()
    for w in range(0, len(english_phoeneme) - batch_size, batch_size):
        _, losses = sess.run([optimizer, loss], 
                             feeding(english_phoeneme[w: w + batch_size], chinese[w: w + batch_size]))
        
        total_loss += losses
        
    total_loss = total_loss / ((len(english_phoeneme) - batch_size) / (batch_size * 1.0))
    LOSS.append(total_loss)
    
    if (q + 1) % 10 == 0:
        print('epoch: ' + str(q + 1) + ', total loss: ' + str(total_loss) + ', s/epoch: ' + str(time.time() - lasttime))

epoch: 10, total loss: 1.12181864582, s/epoch: 95.65810418128967
epoch: 20, total loss: 1.11449022384, s/epoch: 90.3179931640625


KeyboardInterrupt: 